[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/HyperCoast/blob/main/docs/examples/pace_oci_l1.ipynb)

# Visualizing PACE OCI L1 data products with HyperCoast

This notebook demonstrates how to visualize [Plankton, Aerosol, Cloud, ocean Ecosystem (PACE)](https://pace.gsfc.nasa.gov) OCI [L1 data products](https://pace.oceansciences.org/data_table.htm). Part of the notebook is adapted from the NASA OB.DAAC tutorial - [File Structure at Three Processing Levels for the Ocean Color Instrument (OCI)](https://oceancolor.gsfc.nasa.gov/resources/docs/tutorials/notebooks/oci_file_structure/). Credits to the NASA OB.DAAC team for the tutorial.

In [ ]:
# %pip install "hypercoast[extra]"

In [1]:
import hypercoast
import xarray as xr

To download and access the data, you will need to create an Earthdata login. You can register for an account at [urs.earthdata.nasa.gov](https://urs.earthdata.nasa.gov/). Once you have an account, you can run the following cell to search and download PACE OCI L1 data products.

In [2]:
hypercoast.nasa_earth_login()

In [3]:
hypercoast.nasa_earth_login()

short_name = "PACE_OCI_L1B_SCI"
results, gdf = hypercoast.search_nasa_data(
    short_name=short_name,
    bbox=(-90.5642, 29.9749, -89.7143, 30.42),
    temporal=("2024-06-15", "2024-06-16"),
    return_gdf=True,
)

Granules found: 1


In [4]:
gdf.explore()

![](https://i.imgur.com/3FtwIXA.png)

In [ ]:
hypercoast.download_nasa_data(results[0], out_dir="data")

Alternatively, use the following code block to download a sample dataset from [here](https://github.com/opengeos/datasets/releases/tag/hypercoast). 

In [5]:
url = "https://github.com/opengeos/datasets/releases/download/hypercoast/PACE_OCI.20240615T182549.L1B.nc"
filepath = "data/PACE_OCI.20240615T182549.L1B.nc"
hypercoast.download_file(url, filepath)

ChunkedEncodingError: ('Connection broken: IncompleteRead(398458880 bytes read, 1477995760 more expected)', IncompleteRead(398458880 bytes read, 1477995760 more expected))

Let's check the top-level groups in the sample dataset.

In [6]:
hypercoast.netcdf_groups(filepath)

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'data/PACE_OCI.20240615T182549.L1B.nc', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

The top-level groups in the sample dataset are:

```text
['sensor_band_parameters',
 'scan_line_attributes',
 'geolocation_data',
 'navigation_data',
 'observation_data']
```

Let's open the `observation_data` group, which contains the core science variables.

In [ ]:
dataset = xr.open_dataset(filepath, group="observation_data")
print(list(dataset.variables))

The data variables include:

```text
['rhot_blue', 'qual_blue', 'rhot_red', 'qual_red', 'rhot_SWIR', 'qual_SWIR']
```

The dimensions of the `rhot_blue` variable are `("blue_bands", "number_of_scans", "ccd_pixels")`, and it has shape `(119, 1710, 1272)`. The sizes attribute of a variable gives us that information as a dictionary.

In [ ]:
dataset["rhot_blue"].sizes

The dimensions of the `rhot_red` variable are `("red_bands", "number_of_scans", "ccd_pixels")`, and it has shape `(163, 1710, 1272)`

In [ ]:
dataset["rhot_red"].sizes

The dimensions of the `rhot_SWIR` variable are `("SWIR_bands", "number_of_scans", "SWIR_pixels")`, and it has shape `(9, 1710, 1272)`

In [ ]:
dataset["rhot_SWIR"].sizes

Let's plot the reflectance at position `100` in the `blue_bands` dimension.

In [ ]:
plot = dataset["rhot_blue"].sel({"blue_bands": 100}).plot()

![](https://i.imgur.com/EzNMdMG.png)